# #403 Precast multi-span beam and slab bridge with staged construction
<i>Beams are modelled with beam elements and the deck slab is modelled with shell elements. The bearings are offset with rigid links to the underside of the concrete beams.</i>
***

In [ ]:
solve_analyses = False # Run the analysis at the end of the script

Define model parameters

In [ ]:
# Geometric
deck_width = 18
span_lengths = [25,25,25]
bearing_length = 1
no_beams = 4
deck_overhang = 1.5
skew_angle = 15
deck_thk = 0.2
# Meshing
mesh_size = 1

concrete_density = 2.4

# Loading area
carriageway_width = deck_width - 4

# Loads
waterproofing_load = -0.5
surf_load_edge = -2
surf_load_centre = -4
footpath_surface_load_intensity = -1
parapet_uniform_load = -10
# Differential shrinkage strain applied to deck slab only
shrinkage_strain = -0.369764E-3
# Temperatures
temperature_increase = 20
temperature_decrease = -20
# Live loads
pedestrian_live_load = -5
breaking_load_intensity = 5

create_dmi = False
CONST_OFFSET = 10 # Extends curb lines and construction points beyond the model

In [ ]:
# Calculate additional variables
deck_length = sum(span_lengths)
beam_spacing = (deck_width - 2*deck_overhang) / (no_beams-1)
# input checks
assert beam_spacing > 0, "Cant have a negative beam spacing"

carriageway_centrelines = [2*deck_width/8, 6*deck_width/8]


Connect to LUSAS create a new model

In [ ]:
import numpy as np
import math
import sys; sys.path.append('../') # Reference modules in parent directory
from LPI import *
lusas = get_lusas_modeller()
from m100_Tools_And_Helpers import Helpers
Helpers.initialise(lusas)

# if lusas.existsDatabase():
#     raise Exception("This script will create a new model. Please save and close the current model and try again")

# Create a new model
lusas.newProject("Structural", "Precast_Beam_Multi-Span")
# Reference to the model database for convenience
db = lusas.database() 
# 3D model with Z vertical
db.setAnalysisCategory("3D")
db.setVerticalDir("Z")
# Set the unit system
db.setModelUnits("kN,m,t,s,C")

Create the attributes to be assigned to the geometric features. Mesh, Geometric and Material.

In [ ]:
# Beam mesh attributes
beam_mesh_attr = db.createMeshLine(f"Beam {mesh_size=}").setSize("BMI21", mesh_size)
# Beam geometric attribute
beam_geom_attr = db.createGeometricLine("Beam Section").setFromLibrary("UK Sections", "Precast U Beams", "U12", 0, 0, 0)
beam_geom_attr.setEccentricityOrigin("Centroid", "Fibre", "", "A1")
beam_geom_attr.setValue("ez0", deck_thk / 2, 0) # additional eccentricty to account for deck
# get the depth of the beam, to offset the bearings
beam_depth = beam_geom_attr.getValue("zt") - beam_geom_attr.getValue("zb")
support_offset = beam_depth + deck_thk/2
# Diaphragm beam
dia_section_attr = Helpers.create_rectangular_section(db, "Diaphragm", 0.75, beam_depth)
dia_section_attr.setEccentricityOrigin("Centroid", "Fibre", "", "S1")
dia_section_attr.setValue("ez0", deck_thk / 2, 0) # additional eccentricty to account for deck

# Beam material attributes for short and long term
beam_mat_attr = db.createIsotropicMaterial("Concrete beam", 34.8E6, 0.2, concrete_density, 10.0E-6)
beam_mat_attr_creep = db.createIsotropicMaterial("Concrete beam long term", 34.8E6/3, 0.2, concrete_density, 10.0E-6)

# Deck mesh attribute, thick shell elements
deck_mesh_attr    = db.createMeshSurface(f"Shell {mesh_size=}").setRegularSize("QTS4", mesh_size, True)
# Deck surface thickness attribute
deck_surface_attr = db.createGeometricSurface("Deck Slab").setSurface(deck_thk, 0.0)
# Deck surface material attributes for short and long term
deck_mat_attr     = db.createIsotropicMaterial("Concrete deck", 34.8E6, 0.2, concrete_density, 10.0E-6)
deck_mat_attr_creep = db.createIsotropicMaterial("Concrete deck long term", 34.8E6/3, 0.2, concrete_density, 10.0E-6)

# Support attribute
support_fixed_attr = db.createSupportStructural("Fixed").setStructural("R", "R", "R", "R", "F", "F", "F", "F", "C", "F") # F=Free, R=Restrained

# Bearing joint stiffness
joint_mesh_attr = db.createPointMeshElementAttr("Bearing Joint Mesh", "JNT4")
pinned_joint_support_attr = db.createSpringJointMaterial("Fixed Support Stiffness", [1e9, 1e9, 1e9])
pinned_joint_support_attr.setValue("Assignment", "Point")
slide_joint_support_attr = db.createSpringJointMaterial("Slide Support Stiffness", [1, 1e9, 1e9]) # Slide in x direction
slide_joint_support_attr.setValue("Assignment", "Point")

# Specify a local coordinate system that matches the global system to define the joint axes
local_coordinate_attr = db.createLocalCartesianXYAttr("Joint Orientation", 0.0, [0,0,0]).setAxesType("Cartesian")


In order to assign any attributes we need a loadcase to assign them to. <br>We'll create a base analysis with a single loadcase

In [ ]:
base_analysis = db.getAnalysis("Analysis 1")
base_analysis.setName("Base")
# Get the automatically created loadcase in the base analysis and add automatic gravity to it
# The loadcase will be used only to test the model is working correctly
# NOTE: getLoadset and setName function returns a reference to the IFLoadset baseclass and must be cast to IFLoadcase to access the addGravity function
win32.CastTo(db.getLoadset("Loadcase 1", 0).setName("Model Test Gravity"), "IFLoadcase").addGravity(True)

Helper function to calculate beam coordinates accounting for the skew angle

In [ ]:
def skew_x(y:float)->float:
    return y * math.tan(math.radians(skew_angle))

Calculate the coordinates

In [ ]:
# Y Coordinates
# Note the surfaces between the beams are split evenly between the beams
y_crds = [0.0, deck_overhang]
for i in range(0, no_beams-1):
    y_crds.append(y_crds[-1] + beam_spacing/2)
    y_crds.append(y_crds[-2] + beam_spacing)
y_crds.append(deck_width)

# X Coordinates
x_crds = [0]
x = 0
for span_length in span_lengths:
    x_crds.extend([x + bearing_length, x + span_length - bearing_length, x + span_length ])
    x+=span_length

In [ ]:
# Determine the array indices for sub structure locations
support_indices = [0]
for s in span_lengths:
    support_indices.append(support_indices[-1]+3)

Create the model geometry and assign the mesh, geometric and material attributes

In [ ]:
# Create the actual points in the model [X][Y]
deck_points = [[Helpers.create_point(x+skew_x(y), y, 0.0) for y in y_crds]for x in x_crds]             

# Deck surfaces
for iy in range(len(y_crds)-1):
    for ix in range(len(x_crds)-1):
        Helpers.create_surface_from_points([ deck_points[ix][iy], deck_points[ix+1][iy], deck_points[ix+1][iy+1], deck_points[ix][iy+1] ])

# Assign the mesh attribute to the created surfaces
deck_mesh_attr.assignTo("Surfaces")
# Assign the geometric thickness attribute to the created surfaces
deck_surface_attr.assignTo("Surfaces")
# Assign the material attribute to the created surfaces
deck_mat_attr.assignTo("Surfaces")

In [ ]:
long_beam_lines = []
# Longitudinal beams
grp = 1
for iy in range(1, len(y_crds)-1, 2):
    objs = lusas.newObjectSet()
    objs.add( Helpers.get_line_between_points(deck_points[0][iy], deck_points[1][iy]) )
    objs.addColinearNeighbours(1)
    beam_mesh_attr.assignTo(objs)
    beam_geom_attr.assignTo(objs)
    beam_mat_attr.assignTo(objs)
    # Create a group for the current beam
    group = db.createGroup(f"Beam {grp}")
    # Add the line
    group.add(objs)
    # Add the Higher Order Features of the line, which are the associated deck surfaces
    group.addHOF("Surface")
    grp+=1
    long_beam_lines.append(objs.getObjects("Line"))

# Diaphragm beams
for ix in support_indices:
    objs = lusas.newObjectSet()
    objs.add( Helpers.get_line_between_points(deck_points[ix][0], deck_points[ix][1]) )
    objs.addColinearNeighbours(1)
    beam_mesh_attr.assignTo(objs)
    dia_section_attr.assignTo(objs)
    beam_mat_attr.assignTo(objs)

Supports will be modelled at the underside of the precast beams. To do this a joint element is defined represtning the stiffness of the bearing. One end is connceted to the support and the other is connected via a rigid constraint to the plane of the deck

Create two points at the same location between which the bearing joint element will act. Prevent the points from being merged before creating them

In [ ]:
db.getOptions().setBoolean("newFeaturesMergeable", False)

substructure_no = 1
for ix in support_indices:
    support_no = 1
    for iy in range(1, len(y_crds)-1, 2):
        p = deck_points[ix][iy]
        p_constraint = Helpers.create_point(p.getX(), p.getY(), p.getZ() - support_offset)
        p_support    = Helpers.create_point(p.getX(), p.getY(), p.getZ() - support_offset)

        # Create a new attribute for each connection
        rigid_link_attr = db.createRigidLinkConstraint(f"Rigid Link {substructure_no} Support {support_no}")
        # Add the point offset at the support and the corresponding deck point
        objs = lusas.newObjectSet().add(p).add(p_constraint)
        # Get the default assignment object
        assignment = lusas.assignment().setAllDefaults()
        # Assign the rigid links
        rigid_link_attr.assignTo(objs)

        # Assign the support joint elements orientated in global coordinates
        assignment.setLocalCoord(local_coordinate_attr)
        # Add the point offset at the support and the bottom of the rigid link
        objs = lusas.newObjectSet().add(p_support).add(p_constraint)
        # Assign the joint mesh acting between the primary and secondary nodes
        joint_mesh_attr.assignTo(objs)
        # Assign the joint material to the primary node
        if substructure_no == 1:
            pinned_joint_support_attr.assignTo(p_support)
        else:
            slide_joint_support_attr.assignTo(p_support)
        # Assign the support to the primary node
        support_fixed_attr.assignTo(p_support)
        support_no +=1
    substructure_no+=1

db.getOptions().setBoolean("newFeaturesMergeable", True)

Generate the mesh from the assignments

In [ ]:
db.resetMesh()
db.updateMesh()

Create 3 separate analyses, 1st stage is for the beams, 2nd stage is long term and the 3rd stage is short term

In [ ]:
# Deactivate attribute disables elements to which its assigned
deactive_attr = db.createDeactivate("Deact1").setDeactivate("activeMesh", 100.0, 1.0E-6)
# Activate attribute reactivates attributes in the loadcase to which it's assigned
activate_attr = db.createActivate("Act1")

# Create 3 separate analyses, 1st stage is for the beams, 2nd stage is long term and the 3rd stage is short term
stage1_analysis = db.createAnalysisStructural("Phase 1 (Beams only)", False)
stage2_analysis = db.createAnalysisStructural("Phase 2 (Long term)", False)
stage3_analysis = db.createAnalysisStructural("Phase 3 (Short term)", False)

# Loads in Phase 1

In [ ]:
# We'll deactivate the deck in Phase 1 only.
# The deactivate attribute must be assigned to the first loadcase in the analysis, so we'll create that first
beam_sw_loadcase = db.createLoadcase("Beam Self Weight", stage1_analysis.getName())
# Set gravity on the beams only loadcase
beam_sw_loadcase.addGravity(True)

# To assign the attribute to the correct loadcase we need to provide additional information
# We do this with the assignment object
assignment = lusas.assignment().setAllDefaults().setLoadset(beam_sw_loadcase)

deactive_attr.assignTo("Surfaces", assignment)
deactive_attr.assignTo(lusas.newObjectSet().add(dia_section_attr), assignment)
deactive_attr.assignTo(lusas.newObjectSet().add(joint_mesh_attr), assignment)

objs = lusas.newObjectSet()
for iy in range(1, len(y_crds)-1, 2):
    for ix in range(0, len(x_crds)-1, 3):
        objs.add( Helpers.get_line_between_points(deck_points[ix][iy], deck_points[ix+1][iy]) )
        objs.add( Helpers.get_line_between_points(deck_points[ix+2][iy], deck_points[ix+3][iy]) )
        support_fixed_attr.assignTo(deck_points[ix+1][iy], assignment)
        support_fixed_attr.assignTo(deck_points[ix+2][iy], assignment)
deactive_attr.assignTo(objs, assignment)

Calculate and assign loads for the wet concrete of the deck in Phase 1

In [ ]:
# Helper function to get the area of all surfaces attached to the given line
def get_attached_surface_area(line) -> float:
    surface_area = 0
    for obj in line.getHOFs():
        if obj.getTypeCode() == 4: # Surface
                surface_area += obj.getArea()
    return surface_area

In [ ]:
# Create a loadcase for the wet concrete loads to be applied to the beams in Phase 1
wet_conc_loadcase = db.createLoadcase("Wet Concrete", stage1_analysis.getName())
# Set the loadcase in the default assignment object
assignment = lusas.assignment().setAllDefaults().setLoadset(wet_conc_loadcase)

# Now calculate the deck area for each beam based on the attached surfaces
for beam_lines in long_beam_lines:
    for i, line in enumerate(beam_lines):
        deck_area = get_attached_surface_area(line)
        beam_load = round((deck_area * deck_thk * concrete_density)/line.getLineLength(), 3)
        # And create a load attribute for each beam.
        carriageway_surf_load_attr = db.createLoadingBeamDistributed(f"Wet Concrete - {beam_load=}")
        carriageway_surf_load_attr.setBeamDistributed("Parametric", "Global", "beam")
        carriageway_surf_load_attr.addRow(0.0, 0.0, 0.0, -beam_load, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, -beam_load, 0.0, 0.0, 0.0)
        # and assign the load
        carriageway_surf_load_attr.assignTo(line, assignment)

# A similar approach can be followed for all construction and other loads affecting only Phase 1

# Loads in Phase 2

Create loads and loadcases in phase 2 in which the deck elements are activated and materials are considered "long term"

In [ ]:
# Create a loadcase for the parapet loads to be applied to the deck edges in Phase 2
parapet_loads_loadcase = db.createLoadcase("Parapet", stage2_analysis.getName())
# Set the loadcase in the assignment object
assignment = lusas.assignment().setAllDefaults().setLoadset(parapet_loads_loadcase)
# Create a globally distributed load for parapet loads
parapet_load_attr = db.createLoadingGlobalDistributed("Parapet Loads")
parapet_load_attr.setGlobalDistributed("Length", 0.0, 0.0, parapet_uniform_load, 0.0, 0.0, 0.0, 0.0, 0.0, False, 0.0)
# and assign to the two edge longitudinal lines
for iy in [0, len(y_crds)-1]:
    objs = lusas.newObjectSet()
    objs.add( Helpers.get_line_between_points(deck_points[0][iy], deck_points[1][iy]) )
    objs.addColinearNeighbours(1)
    parapet_load_attr.assignTo(objs, assignment)

In [ ]:
# Create a loadcase for the Waterproofing loads to be applied to the entire deck surface in Phase 2
waterproofing_loadcase = db.createLoadcase("Waterproofing", stage2_analysis.getName())
# Set the loadcase in the assignment object
assignment = lusas.assignment().setAllDefaults().setLoadset(waterproofing_loadcase)
# Create a locally distributed load for Waterproofing loads
wp_load_attr = db.createLoadingLocalDistributed("Waterproofing").setLocalDistributed(0.0, 0.0, waterproofing_load, "surface")
# and assign the deck surfaces (using the surfaces set created early)
wp_load_attr.assignTo("Surfaces", assignment)

Represent the variation of carriageway surfacing as a discrete load which is projected onto the geometry and therefore is not restricted to the shape of the geometry features

In [ ]:
# Calculate the maximum x coordinate of the deck accounting for the skew
max_x = deck_length+skew_x(deck_width)
# Y coordinate of the first footpath (this will be the width since the deck is positioned at 0,0,0)
footway1_y = (deck_width - carriageway_width)/2
# Y coordinate of the second footpath
footway2_y = deck_width - footway1_y
# Y coordinate of the deck centreline
centre_y = footway1_y+(footway2_y - footway1_y)/2

# Create a Discrete 8 node patch load to model the variation in surfacing thickness from the centre to the edge of the carriageway
carriageway_surf_load_attr = db.createLoadingDiscretePatch("Carriageway Surface").setDiscretePatch("surf8", "Z")
# Add the points of an 8 node patch load in which the load intensity varies for the edges to the centre
# Set the z coordinate to be the top of the deck slab for better visualisation
carriageway_surf_load_attr.addRow(0.0,     footway1_y,   deck_thk, surf_load_edge)
carriageway_surf_load_attr.addRow(0.0,     deck_width/2, deck_thk, surf_load_centre)
carriageway_surf_load_attr.addRow(0.0,     footway2_y,   deck_thk, surf_load_edge)
carriageway_surf_load_attr.addRow(max_x/2, footway2_y,   deck_thk, surf_load_edge)
carriageway_surf_load_attr.addRow(max_x,   footway2_y,   deck_thk, surf_load_edge)
carriageway_surf_load_attr.addRow(max_x,   deck_width/2, deck_thk, surf_load_centre)
carriageway_surf_load_attr.addRow(max_x,   footway1_y,   deck_thk, surf_load_edge)
carriageway_surf_load_attr.addRow(max_x/2, footway1_y,   deck_thk, surf_load_edge)

# Create a loadcase for the Surfacing loads to be applied to the carriageway area of the deck
surfacing_loadcase = db.createLoadcase("Surfacing", stage2_analysis.getName())
# Set the loadcase in the assignment object
assignment = lusas.assignment().setAllDefaults().setLoadset(surfacing_loadcase)
# For discrete loads we specify some additional settings
# For loads that extend beyond the structure, tell LUSAS to ignore them
assignment.setLoadMoving("Exclude All Load")
# The type of model we want to project onto is an area, this is the deck slab surface area
assignment.setSearchAssignType("area")

# Now assign the load attribute to the point at the origin, which is at the start of first longitudinal line
point1 = deck_points[0][0]
carriageway_surf_load_attr.assignTo(point1, assignment)

In [ ]:
# Create lines to represent the carriageway kerbs that can be used for the Vehicle Load Optimiser
curb_line1 = Helpers.create_line([-CONST_OFFSET, footway1_y, 0.5], [CONST_OFFSET + deck_length+skew_x(deck_width), footway1_y, 0.5])
curb_line2 = Helpers.create_line([-CONST_OFFSET, footway2_y, 0.5], [CONST_OFFSET + deck_length+skew_x(deck_width), footway2_y, 0.5])
# Set the colour to distinguish them from model lines
curb_line1.setPen(2)
curb_line2.setPen(2)

Discrete loads can be be re-used in the model and assigned to a different origin. We'll do that for the footpath loads.<br/>
We'll make a generic function to create a 4 noded discrete patch load the size of the footpath.

In [ ]:
def create_footpath_load(name:str, load1:float, load2:float)->'IFLoadingDiscretePatch':
    attr = db.createLoadingDiscretePatch(name).setDiscretePatch("surf4", "Z")
    # Add the points of an 4 node patch load in which the load intensity varies across the width
    # The x coordinates are increased by 1 because we'll assign these loads to points created at x=-1 to hold these loads
    attr.addRow(CONST_OFFSET,       0.0,        deck_thk, load1)
    attr.addRow(CONST_OFFSET,       footway1_y, deck_thk, load2)
    attr.addRow(CONST_OFFSET+max_x, footway1_y, deck_thk, load2)
    attr.addRow(CONST_OFFSET+max_x, 0.0,        deck_thk, load1)
    # Explicitly specify how the patch load should be divided up.
    # We know the width of the footway is small so we dont need so many divisions
    # Limiting the divisions speeds up the time to resolve the patch load into nodal loads
    # One load per element is sufficient
    attr.setDivisions(math.ceil(footway1_y*mesh_size), math.ceil(deck_length*mesh_size))
    return attr

In [ ]:
# Create a surfacing load using the generic function
footpath_surf_load_attr = create_footpath_load("Footpath Surface", footpath_surface_load_intensity, footpath_surface_load_intensity)

# Assign this to a new point using the assignment object set up for the deck surfacing loads
footpath_point1 = Helpers.create_point(-CONST_OFFSET, 0.0, 0.0)
footpath_point1.setPen(2)
footpath_surf_load_attr.assignTo(footpath_point1, assignment)
# And again to a 2nd new point
footpath_point2 = Helpers.create_point(-CONST_OFFSET, footway2_y, 0.0)
footpath_point2.setPen(2)
footpath_surf_load_attr.assignTo(footpath_point2, assignment)

# Note that we set the colour of the new points to blue so we can identify that they are not part of the model and only used to locate the discrete loads

Create a differential shrinkage strain load acting only on the deck elements

In [ ]:
# Create an initial stress/strain attribute and populate only the strains Ex,Ey at the top middle and bottom of the shell
# The indices of these components are specified in the list, all other values are zero
strain_array = [0]*30
for i in [5,6,15,16,25,26]:
    strain_array[i] = shrinkage_strain

# Create the attribute
shrink_attr = db.createLoadingStressStrain("Shrinkage strain").setStressStrain("Initial", strain_array, "Surface", "Thick shell", 3)

# Create a loadcase for the shrinkage loads to be applied to the deck in Phase 2
shrink_loadcase = db.createLoadcase("Deck Shrinkage", stage2_analysis.getName())
# Set the loadcase in the assignment object
assignment = lusas.assignment().setAllDefaults().setLoadset(shrink_loadcase)
# Use the surface set defined early to assign the load to all deck surfaces
shrink_attr.assignTo("Surfaces", assignment)

Phase 2 represents a long term stage so in this analysis we'll consider creep adjusted materials.<br/>Materials must be assigned to the first loadcase in a linear analysis

In [ ]:
# Find the first loadcase in Analysis "Phase 2"
first_stage_2_loadcase = stage2_analysis.getLoadcases()[0]
# Assign the materials to the first loadcase
assignment = lusas.assignment().setAllDefaults().setLoadset(first_stage_2_loadcase)
beam_mat_attr_creep.assignTo(lusas.newObjectSet().add(beam_mesh_attr), assignment)
deck_mat_attr_creep.assignTo("Surfaces", assignment)

# Loads in Phase 3<br/>

Create temperature loads

In [ ]:
# Create a loadcases for temperature
temp_warm_loadcase = db.createLoadcase("Temperature Gradient Warming", stage3_analysis.getName())
temp_cool_loadcase = db.createLoadcase("Temperature Gradient Cooling", stage3_analysis.getName())
# Temperature loading warming 
temp_warm_attr = db.createLoadingTemperature("Gradient Warming").setTemperature("element", temperature_increase, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
# Temperature loading cooling 
temp_cool_attr = db.createLoadingTemperature("Gradient Cooling").setTemperature("element", temperature_decrease, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
# Assign the temperature warming load
assignment = lusas.assignment().setAllDefaults().setLoadset(temp_warm_loadcase)
temp_warm_attr.assignTo("Surfaces", assignment)
# Assign the temperature cooling load
assignment = lusas.assignment().setAllDefaults().setLoadset(temp_cool_loadcase)
temp_cool_attr.assignTo("Surfaces", assignment)

In [ ]:
# Create a loadcases for temperature
temp_warm_loadcase = db.createLoadcase("Temperature Uniform Warming", stage3_analysis.getName())
temp_cool_loadcase = db.createLoadcase("Temperature Uniform Cooling", stage3_analysis.getName())
# Temperature loading warming 
temp_warm_attr = db.createLoadingTemperature("Uniform Warming").setTemperature("element", temperature_increase, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
# Temperature loading cooling 
temp_cool_attr = db.createLoadingTemperature("Uniform Cooling").setTemperature("element", temperature_decrease, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
# Assign the temperature warming load
assignment = lusas.assignment().setAllDefaults().setLoadset(temp_warm_loadcase)
temp_warm_attr.assignTo("Surfaces", assignment)
temp_warm_attr.assignTo(lusas.newObjectSet().add(beam_mesh_attr), assignment)
# Assign the temperature cooling load
assignment = lusas.assignment().setAllDefaults().setLoadset(temp_cool_loadcase)
temp_cool_attr.assignTo("Surfaces", assignment)
temp_cool_attr.assignTo(lusas.newObjectSet().add(beam_mesh_attr), assignment)

Pedestrian live loads

In [ ]:
# We can reuse the discrete patch load function to create pedestrian live loads 
ped_live_load_attr = create_footpath_load("Pedestrian Live load", pedestrian_live_load, pedestrian_live_load)

# Create a loadcase for the pedestrian live loads
ped_loads_loadcase = db.createLoadcase("Pedestrian live loads", stage3_analysis.getName())

# Redefine the assignment object for the discrete load assignment as we did for the surfacing loads
assignment = lusas.assignment().setAllDefaults().setLoadset(ped_loads_loadcase)
assignment.setLoadMoving("Exclude All Load")
assignment.setSearchAssignType("area")

# Again, assign the same load to the two dedicated points
ped_live_load_attr.assignTo(footpath_point1, assignment)
ped_live_load_attr.assignTo(footpath_point2, assignment)

Breaking load

In [ ]:
for i, y in enumerate(carriageway_centrelines):
    # Create a discrete patch load with the loads acting in the X direction.
    # Patch loads are projected onto the structure in the direction of their normal. Which will be the Z axis in this case.
    breaking_load_attr = db.createLoadingDiscretePatch(f"Breaking load lane {i+1}").setDiscretePatch("surf4", "X")
    # Add the points of an 4 node patch load (note assume lane width = 3.5m)
    breaking_load_attr.addRow(0.0,   y + 1.75, deck_thk, breaking_load_intensity)
    breaking_load_attr.addRow(0.0,   y - 1.75, deck_thk, breaking_load_intensity)
    breaking_load_attr.addRow(max_x, y - 1.75, deck_thk, breaking_load_intensity)
    breaking_load_attr.addRow(max_x, y + 1.75, deck_thk, breaking_load_intensity)
    breaking_load_attr.setDivisions(math.ceil(3.5*mesh_size), math.ceil(deck_length*mesh_size))
    # Create a loadcase for the braking loads
    breaking_loads_loadcase = db.createLoadcase(f"Breaking load lane {i+1}", "Phase 3")

    # Redefine the assignment object for the discrete load assignment as we did for the surfacing loads
    assignment = lusas.assignment().setAllDefaults().setLoadset(breaking_loads_loadcase)
    assignment.setLoadMoving("Exclude All Load")
    assignment.setSearchAssignType("area")

    # Assign to the point at the origin
    breaking_load_attr.assignTo(point1, assignment)

Create reference paths for moving loads

In [ ]:
for i, y in enumerate(carriageway_centrelines):
    ref_path = db.createReferencePath(f"Lane {i+1}")
    ref_path.getDefn().addStraightV(0.0, y, 0.0, max_x, y, 0.0)
    ref_path.getDefn().setFacetData(20, 6, -1.0, 3.0, -1.0)
    ref_path.getDefn().setSmoothing(False)
    ref_path.getDefn().setFilletType(0, True)
    ref_path.setDistanceStartOfPath(0.0)
    ref_path.setTransverseDirection("Perpendicular")
    ref_path.updateDefinition()

In [ ]:
# Create a search area attribute for just the nodes in the deck slab
search_area_attr = db.createSearchArea("Deck Slab")
# Create a set of objects with the deck surface geometric attribute assigned to them
surfaces = lusas.newObjectSet().add(deck_surface_attr)
# Assign the search area attribute to the same surfaces
search_area_attr.assignTo(surfaces, lusas.assignment().setAllDefaults())

Create a discrete load attribute to represent the axle loads of a T44 vehicle

In [ ]:
#Create a discrete load attribute which defines a group of point loads
vehicle_attr = db.createLoadingDiscretePoint("T44")
# Specify the load direction (Z), the projection vector (Z)
vehicle_attr.setDiscrete("Z", [0,0,1])
# Add point loads for each wheel of the vehicle
vehicle_attr.addRow(-4.55, -0.9, 0.0, -24.0)
vehicle_attr.addRow(-4.55, 0.9, 0.0, -24.0)
vehicle_attr.addRow(-0.85, -1.0, 0.0, -24.0)
vehicle_attr.addRow(-0.85, -0.8, 0.0, -24.0)
vehicle_attr.addRow(-0.85, 0.8, 0.0, -24.0)
vehicle_attr.addRow(-0.85, 1.0, 0.0, -24.0)
vehicle_attr.addRow(0.35, -1.0, 0.0, -24.0)
vehicle_attr.addRow(0.35, -0.8, 0.0, -24.0)
vehicle_attr.addRow(0.35, 0.8, 0.0, -24.0)
vehicle_attr.addRow(0.35, 1.0, 0.0, -24.0)
vehicle_attr.addRow(3.35, -1.0, 0.0, -24.0)
vehicle_attr.addRow(3.35, -0.8, 0.0, -24.0)
vehicle_attr.addRow(3.35, 0.8, 0.0, -24.0)
vehicle_attr.addRow(3.35, 1.0, 0.0, -24.0)
vehicle_attr.addRow(4.55, -1.0, 0.0, -24.0)
vehicle_attr.addRow(4.55, -0.8, 0.0, -24.0)
vehicle_attr.addRow(4.55, 0.8, 0.0, -24.0)
vehicle_attr.addRow(4.55, 1.0, 0.0, -24.0)


Create a moving load analysis for the vehicle

In [ ]:
# Create a moving load analysis which uses the T44 Vehicle defined above
analysis = db.createAnalysisStaticMovingLoad("Phase 3 - Moving load lane 1")
# The moving load with follow the reference path named "Lane 1"
analysis.setStaticMovingLoadAnalysis("T44", "Lane 1", "Deck Slab", "area", "All", "3D", "horizontal", "Exclude All Load", "Division number", 10, 0.0, "Reverse", True)
analysis.setDefinitionMenuID(837, None ,True)

Create beam/shell slicing to compute equivalent beam results. Slices are positioned at 2m increments for demo

In [ ]:
slice_type = 4      # Constant spacing
slice_distances = "2"
include_points = False
start_distance = 0
about_neutral_axis = True
use_effective_width = False         # Use groups instead
effective_width = beam_spacing
whole_elements = False
smooth_corners = True
extent = 2 # Group
rotation = 0

# Create a beam/shell slice utility for each beam
for i, lines in enumerate(long_beam_lines):
    slice = db.createBeamShellSlice(f"Beam {i+1}")
    line_ids = ";".join( [str(line.getID()) for line in lines] )
    slice.setBeamShellSlice(slice_type, slice_distances, include_points, start_distance, about_neutral_axis, use_effective_width, 
                            effective_width, whole_elements, smooth_corners, "Slice", extent, f"Beam {i+1}", line_ids, rotation)
    
# Turn of display of the beam/shell slice planes
lusas.view().utilities().setBeamShellSliceOptions(False, False, False, False)

lusas.enableTrees(1)

Influence analysis

In [ ]:
if create_dmi:
    # Create a Direct Method Influence Analysis
    dmi_analysis = db.createAnalysisDirectMethodInf("Phase 3 - DMI")
    # dmi_analysis.setLoadDirection("Vertical")
    # dmi_analysis.setLoadMagnitude(1.0E3)
    # Specify the elements in the search area deck slab
    dmi_analysis.setSearchArea("Deck Slab")
    dmi_analysis.setSearchAssignType("area")
    dmi_analysis.setIsGrillageType(1)
    dmi_analysis.setGridCentreline("")
    dmi_analysis.setIncludeMidSideNodes(False)
    dmi_analysis.createElements()
    dmi_analysis.setSelectedResultsGroup("assignments")
    dmi_analysis.setSelectedElementOutputGroup("all")
    dmi_analysis.setSelectedNodeOutputGroup("all")


In [ ]:
if create_dmi:
    # Create an influence attribute to envelope all locations
    attr = db.createInfluenceEnvelope("DMI Influence for Bending")
    # The influence shapes will be based on the integrated slices resultants of beams and shells 
    attr.setEntity("Beam/Shell Slice Resultants")
    # Consider the major axis bending moment
    attr.addComponentNames(["My"])
    # And include coincident effects
    attr.includeCoincident(True)

    # Assign the influence attribute to all beam shell slices
    for i, lines in enumerate(long_beam_lines):
        attr.assignToBeamShellSlice(f"Beam {i+1}", "Phase 3 - DMI")

Create some simple post-processing combinations and envelopes

In [ ]:
# Create a basic combination for dead loads
dl_combination = db.createCombinationBasic("DL", "Structural", 0)
dl_combination.addEntry(1.0, beam_sw_loadcase)
dl_combination.addEntry(1.0, wet_conc_loadcase)

# Another combination for super-imposed dead loads
sdl_combination = db.createCombinationBasic("SDL", "Structural", 0)
sdl_combination.addEntry(1.0, parapet_loads_loadcase)
sdl_combination.addEntry(1.0, waterproofing_loadcase)
sdl_combination.addEntry(1.0, surfacing_loadcase)

In [ ]:
# Create an envelope of the warming and cooling temperature loadcases
envelope = db.createEnvelope("Thermal", "Structural", 0, 0)
envelope.addEntry(temp_warm_loadcase)
envelope.addEntry(temp_cool_loadcase)

In [ ]:
combination = db.createCombinationSmart("SLS", "Structural", 0, 0)
combination.addEntry(1.0, 0.0, dl_combination)
combination.addEntry(1.0, 0.0, sdl_combination)
combination.addEntry(0.0, 1.0, shrink_loadcase)
# For envelopes add both the min and max versions of the loadset. 
# The program will consider the correct max or min loadset corresponding to the active parent loadset
combination.addEntry(0.0, 1.0, envelope)
combination.addEntry(0.0, 1.0, envelope.getAssocLoadset())

# Getting and viewing analysis results (load effects)

Solve the analyses

In [ ]:
# Solve each analysis and open all the available results files
if solve_analyses:
    for analysis in db.getAnalyses():
        analysis.solve(True)
    db.openAllResults(False)

Plot diagrams of the beam shell resultants

In [ ]:
if solve_analyses:
    lusas.view().insertDiagramsLayer()
    diagrams_layer = lusas.view().diagrams()
    diagrams_layer.setResultsTransformNone()
    diagrams_layer.setResults("Beam/Shell Slice Resultants", "My")
    diagrams_layer.setLocation("BeamShellSlice")
    diagrams_layer.setAssocOption("Single")

Show the reactions in the values layer

In [ ]:
lusas.view().insertValuesLayer()
lusas.view().values().setResultsTransformNone()
lusas.view().values().setResults("Reaction", "FZ")

Set the view to be perspective mode

In [ ]:
lusas.view().setProjection(1)
lusas.view().setRotationMatrix(0.5, 0.25, -0.8, 0.0, -0.8, 0.2, -0.5, 0.0, 0.0, 0.9, 0.3, 0.0, 0.0, 0.0, 0.0, 1.0)
lusas.view().setScaledToFit(True)

# set the pedestrian live load loadcase active
lusas.view().setActiveLoadset(ped_loads_loadcase)

Set the view to display the assigned loading in the attributes layers as well as the geometric "fleshing"

In [ ]:
lusas.view().insertAttributesLayer()
lusas.view().attributes().drawPatchByDefinition(True)
lusas.view().attributes().visualiseAll("Loading")
lusas.view().attributes().visualiseAll("Geometric")
#lusas.view().attributes().visualiseAllTransparent("Geometric")
lusas.view().attributes().setShowDeformed("Geometric", True)

Create results tables known as "Print Results Wizards"

In [ ]:
results_wizard_utility = db.createPrintResultsWizard("Beam Resultants")

results_wizard_utility.setResultsContent("Tabular")                       # Tabular results only, summary not required
results_wizard_utility.setResultsEntity("Beam/Shell Slice Resultants")
results_wizard_utility.setResultsLocation("BeamShellSlice")

# Specify the loadcases required.
results_wizard_utility.setLoadcasesOption("Selected")
lcIDs = [1]
# We need to pass corresponding array of results ids
lcResFileIDs = [0] *len(lcIDs)
lcEigenvalueIDs = [-1] *len(lcIDs) # No eigenvalues hence -1
lcHarmonicIDs = [-1] *len(lcIDs)   # No harmonic loadcases hence -1
results_wizard_utility.setLoadcases(lcIDs, lcResFileIDs, lcEigenvalueIDs, lcHarmonicIDs)

# Results components to report
results_wizard_utility.setComponents(["Fx", "Fy", "Fz", "Mx", "My", "Mz"])
 # Add the coordinates of each slice location for use with design checks, plots etc
results_wizard_utility.showCoordinates(True)